In [284]:
import pandas as pd
import time
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup

#Y40B10A.2a.1
#F54H12.1c.3

In [285]:
'''
    import pandas as pd
    import time
    import json
    import os
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    import requests
    from bs4 import BeautifulSoup

    #Y40B10A.2a.1
    #F54H12.1c.3

    #json 靜態爬
    transcript_name =request.POST['transcript_id'] #由transcript.js回傳
    response = requests.get(
            "https://wormbase.org/rest/widget/transcript/" + transcript_name + "/sequences",
            headers={
                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
            })

    soup = BeautifulSoup(response.text, "html.parser")
    data= json.loads(str(soup))

    start = []
    stop = []
    #處理變成dict
    strand = data["fields"]['unspliced_sequence_context']['data']['strand']
    #check + -
    if strand == "+" :
        data1 = data["fields"]['unspliced_sequence_context']['data']['positive_strand']['sequence']
        data2 = data["fields"]['spliced_sequence_context']['data']['positive_strand']['sequence']
        for i in range(0,len(data2)-1):
            if (data2[i].islower() & data2[i+1].isupper()):
                start.append(i+2)
            elif (data2[i].isupper() & data2[i+1].islower()):
                stop.append(i+1)
        if stop == []:
            stop.append(len(data2))
    else:
        data1 = data["fields"]['unspliced_sequence_context']['data']['negative_strand']['sequence']
        data2 = data["fields"]['spliced_sequence_context']['data']['negative_strand']['sequence']
        for i in range(0,len(data2)-1):
            if (data2[i].islower() & data2[i+1].isupper()):
                start.append(i+2)
            elif (data2[i].isupper() & data2[i+1].islower()):
                stop.append(i+1)
        if stop == []:
            stop.append(len(data2))

    #靜態爬

    #check + -
    if strand == "+" :
        transcriptdata1 = data["fields"]['unspliced_sequence_context']['data']['positive_strand']['features']
        transcriptdata2 = data["fields"]['spliced_sequence_context']['data']['positive_strand']['features']
       
    else:
        transcriptdata1= data["fields"]['unspliced_sequence_context']['data']['negative_strand']['features']
        transcriptdata2= data["fields"]['spliced_sequence_context']['data']['negative_strand']['features']

    def count_len(start: int,end: int):
        return end-start +1

    unspliced = pd.DataFrame(columns = ['type','start','stop'])
    spliced = pd.DataFrame(columns = ['type','start','stop'])

    for i in range(len(transcriptdata1)):
        unspliced.loc[i] = transcriptdata1[i]
    
    for i in range(len(transcriptdata2)):
        spliced.loc[i] = transcriptdata2[i]


    CDS = {
    'type': 'CDS',
    'start':start[len(start)-1],
    'stop':stop[len(stop)-1]}

    CDS = pd.DataFrame([CDS])
    spliced = pd.concat([spliced, CDS], ignore_index=True)

    unspliced['length'] = unspliced.apply(lambda x: count_len(x['start'],x['stop']),axis=1)
    spliced['length'] = spliced.apply(lambda x: count_len(x['start'],x['stop']),axis=1)
    
    #重新命名
    #unspliced 
    count_Intron_u= 1
    count_Exon_u = 1
    for k in range(0,len(unspliced)):
        if unspliced.at[k,'type'] == 'intron':
            unspliced.at[k,'type'] = 'Intron' + str(count_Intron_u)
            count_Intron_u+= 1
        elif unspliced.at[k,'type'] == 'exon':
            unspliced.at[k,'type'] = 'Exon' + str(count_Exon_u)
            count_Exon_u += 1
        elif unspliced.at[k,'type'] == 'five_prime_UTR':
            unspliced.at[k,'type'] = '5\'UTR'
        elif unspliced.at[k,'type'] == 'three_prime_UTR':
            unspliced.at[k,'type'] = '3\'UTR'
    #spliced
    count_Exon_s = 1
    for k in range(0,len(spliced)):
        if spliced.at[k,'type'] == 'exon':
            spliced.at[k,'type'] = 'Exon' + str(count_Exon_s)
            count_Exon_s += 1
        elif spliced.at[k,'type'] == 'five_prime_UTR':
            spliced.at[k,'type'] = '5\'UTR'
        elif spliced.at[k,'type'] == 'three_prime_UTR':
            spliced.at[k,'type'] = '3\'UTR'

    #for answer
    #unspliced       
    #print('unspliced:\n',unspliced)
    #spliced
    #print('spliced:\n',spliced)
    

    translation = data['fields']['protein_sequence']['data']['sequence']
    #print('translation:',translation)
             
'''

'\n    import pandas as pd\n    import time\n    import json\n    import os\n    from selenium import webdriver\n    from selenium.webdriver.common.by import By\n    import requests\n    from bs4 import BeautifulSoup\n\n    #Y40B10A.2a.1\n    #F54H12.1c.3\n\n    #json 靜態爬\n    transcript_name =request.POST[\'transcript_id\'] #由transcript.js回傳\n    response = requests.get(\n            "https://wormbase.org/rest/widget/transcript/" + transcript_name + "/sequences",\n            headers={\n                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"\n            })\n\n    soup = BeautifulSoup(response.text, "html.parser")\n    data= json.loads(str(soup))\n\n    start = []\n    stop = []\n    #處理變成dict\n    strand = data["fields"][\'unspliced_sequence_context\'][\'data\'][\'strand\']\n    #check + -\n    if strand == "+" :\n        data1 = data["fields"][\'unspliced_sequence_context\'][\'data\'][\'positive_strand\'][

In [286]:
'''
#for 動態爬蟲
transcript_name = input("Enter the transcript name :")
url = "https://wormbase.org/species/c_elegans/transcript/" + transcript_name +"#06--10"
time.sleep(2)

#Y40B10A.2a.1
#F54H12.1c.3


# 動態crawler 
driver = webdriver.Chrome() 
driver.get(url)
time.sleep(2)
button1 = driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[3]/div[2]/div[1]/li[12]/div/div[2]/div[2]/div[1]/div[2]/div/div/div[1]/div/button[2]').click()
button2 = driver.find_element(By.XPATH,'/html/body/div[2]/div[3]/div[3]/div[2]/div[1]/li[12]/div/div[2]/div[2]/div[1]/div[2]/div/div/div[2]/div/button[2]').click()
time.sleep(2)
driver.close()


#unspliced
with open('/home/cosbi/Downloads/unspliced+UTRTranscriptSequence_' + transcript_name + '.fasta','r') as file:
    data1 = file.readlines()
#spliced
with open('/home/cosbi/Downloads/spliced+UTRTranscriptSequence_' + transcript_name + '.fasta','r') as file1:
    data2 = file1.readlines()

print(data1)
print(data2)
'''

'\n#for 動態爬蟲\ntranscript_name = input("Enter the transcript name :")\nurl = "https://wormbase.org/species/c_elegans/transcript/" + transcript_name +"#06--10"\ntime.sleep(2)\n\n#Y40B10A.2a.1\n#F54H12.1c.3\n\n\n# 動態crawler \ndriver = webdriver.Chrome() \ndriver.get(url)\ntime.sleep(2)\nbutton1 = driver.find_element(By.XPATH,\'/html/body/div[2]/div[3]/div[3]/div[2]/div[1]/li[12]/div/div[2]/div[2]/div[1]/div[2]/div/div/div[1]/div/button[2]\').click()\nbutton2 = driver.find_element(By.XPATH,\'/html/body/div[2]/div[3]/div[3]/div[2]/div[1]/li[12]/div/div[2]/div[2]/div[1]/div[2]/div/div/div[2]/div/button[2]\').click()\ntime.sleep(2)\ndriver.close()\n\n\n#unspliced\nwith open(\'/home/cosbi/Downloads/unspliced+UTRTranscriptSequence_\' + transcript_name + \'.fasta\',\'r\') as file:\n    data1 = file.readlines()\n#spliced\nwith open(\'/home/cosbi/Downloads/spliced+UTRTranscriptSequence_\' + transcript_name + \'.fasta\',\'r\') as file1:\n    data2 = file1.readlines()\n\nprint(data1)\nprint(data2)\n

In [287]:
#json 靜態爬
transcript_name = input("Enter the transcript name :")
response = requests.get(
        "https://wormbase.org/rest/widget/transcript/" + transcript_name + "/sequences",
        headers={
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        })

soup = BeautifulSoup(response.text, "html.parser")
data= json.loads(str(soup))

#處理變成dict
strand = data["fields"]['unspliced_sequence_context']['data']['strand']
#check + -
if strand == "+" :
    data1 = data["fields"]['unspliced_sequence_context']['data']['positive_strand']['sequence']
    data2 = data["fields"]['spliced_sequence_context']['data']['positive_strand']['sequence']
else:
    data1 = data["fields"]['unspliced_sequence_context']['data']['negative_strand']['sequence']
    data2 = data["fields"]['spliced_sequence_context']['data']['negative_strand']['sequence']
print(data1)
print(data2)

aagagacgcagaaatcacaatgtttgtcatcgtttttttcttgttctctcggactatctgatagtataaattcttccctcttttgcctccccaatccaattaattttcagtttttttttccgaaaaaaaaaATGTCCAACGTCGCAAAAAGCTACAAGAGCTCGGATCCAGTGATTGCGTATTGCTCGGAGgtttgtagtttgtagtttgtagtttgtaatgtcacggatttctggtttccttcataaattgaaatggaagagtttgccgaactaggccattttggctcggccatatctgttgtagatttacggcgcgttgcgtgtcgcgtcgcggttcgagtttagttgttaaactaaatgtatttgtccgtgtggagtacacgactttcccacgcgttgtccggcgggcgattgtcaatggagcgcgaaaaattcaatgaggaaggcccacggtagcacagaaactagatgggttggtttatgatactcagtctggaaacctatattggctactatctccaaaaccatcacaaaatcgattttgcgatgcatataagaaatgaccgcaatgaaattatctatctttatttgtgatgaattttcgattttgtacttcctggccaagttatgcacgtttgttcgatggagcgcgtttgaacccatctagcaactgaaccaccgtgggccagaaccccccgtgaatcgtcgtggaaacgcgctccgcctccaacagttgggtctcactaggtaaaaatttgggcggcaaaattgtaaattcaaatttttttaccaattttcaagcattttcttgaagtttcagaagaaaaataaaataaacatttttttttcgatttttcaatattgcaaattcacataattctaaaactcgtacttaaaggcgcatggagttattcggatgggtctcaccgcgaaaaatgcccttttttacctcctccggtgctcgttttttatagggacgcagagcaaactcgccctaa

In [288]:
c = 0

def check(a,b):
    transcription = ''  # 空的list
    i = 0
    while i < len(a):
        # 找Exon
        if a[i].isupper():  # 判斷大寫
            transcription += a[i]  # 把該個字元加進去
            if i == len(a)-1:  # 用來停止while迴圈
                break
            elif a[i+1].islower():
                transcription += ','
        # 找Intron
        elif a[i].islower():  # 判斷小寫
            transcription += a[i]
            if i == len(a)-1:
                break
            elif a[i+1].isupper():
                transcription += ','
        i += 1  # 逐個檢查
    # 弄成DataFrame的形式
    df1 = pd.DataFrame(columns=['Exon&Intron'])  # 建立兩個column的DataFrame
    df1['Exon&Intron'] = transcription.split(',')
    transcription = '' #空的list
    i = 0 

    while i <len(b):
        #找Exon
        if b[i].isupper(): #判斷大寫
            transcription  += b[i] #把該個字元加進去
            if i == len(b)-1: #用來停止while迴圈
                break
            elif b[i+1].islower():
                transcription  += ','
        #找Intron
        elif b[i].islower(): #判斷小寫
            transcription  += b[i]
            if i == len(b)-1:
                break
            elif b[i+1].isupper():
                transcription  += ','
        i += 1 #逐個檢查
    #弄成DataFrame的形式
    df2 = pd.DataFrame(columns=['Exon&Intron']) #建立兩個column的DataFrame
    df2['Exon&Intron'] = transcription.split(',')

    global c
    if df1.at[0,'Exon&Intron'] == df2.at[0,'Exon&Intron']:
        c = 0
        print('easy')

    else:
        c = 1 
        print('hard')





In [289]:
#check 5'UTR 是否一樣
check(data1,data2)

easy


In [290]:
if c == 0:
    print('easy')
    #把大小寫切出來
    
    #for unspliced
    transcription = '' #空的list
    upper_lower = ''  #空的list 用來記錄有幾次大小寫變化
    i = 0 

    while i <len(data1):
        #找Exon
        if data1[i].isupper(): #判斷大寫
            transcription  += data1[i] #把該個字元加進去
            if i == len(data1)-1: #用來停止while迴圈
                break
            elif data1[i+1].islower():
                transcription  += ','
                upper_lower += 'U,'
        #找Intron
        elif data1[i].islower(): #判斷小寫
            transcription  += data1[i]
            if i == len(data1)-1:
                break
            elif data1[i+1].isupper():
                transcription  += ','
                upper_lower += 'L,'
        i += 1 #逐個檢查
    #處理最後一個的大小寫
    if data1[len(data1)-2].isupper():
        upper_lower +='U'
    else:
        upper_lower +='L'

    #弄成DataFrame的形式
    df1 = pd.DataFrame(columns=['temp','Exon&Intron']) #建立兩個column的DataFrame
    df1['temp'] = upper_lower.split(',') #split 表示依照什麼東西來區別 因為是string
    df1['Exon&Intron'] = transcription.split(',')

    #處理5'UTR和3'UTR
    if df1.at[0,'temp'] == 'L': #開頭小寫表示有5'UTR
        df1.at[0,'temp'] = '5'
    if df1.at[len(df1)-1,'temp'] == 'L': #結尾小寫表示有3'UTR
        df1.at[len(df1)-1,'temp'] = '3'
    #print(df1)
    

    #判斷Intron(小寫的)跟Exon(大寫的)
    count_Intron= 1
    count_Exon = 1
    for k in range(0,len(df1)):
        if df1.at[k,'temp'] == 'L':
            df1.at[k,'Name'] = 'Intron' + str(count_Intron)
            count_Intron+= 1
        elif df1.at[k,'temp'] == 'U':
            df1.at[k,'Name'] = 'Exon' + str(count_Exon)
            count_Exon += 1
        elif df1.at[k,'temp'] == '5':
            df1.at[k,'Name'] = '5\'UTR'
        elif df1.at[k,'temp'] == '3':
            df1.at[k,'Name'] = '3\'UTR'
    #計算Length
    #弄一個dataframe 給spliced 用
    df3 = pd.DataFrame()
    df3 = df1
    for k in range(0,len(df3)):
        if df3.at[k,'temp'] == 'L' :
            df3 = df3.drop(labels = k)
    df3 = df3.reset_index()  
    df3 = df3.drop(columns='index')

    df1['Length'] = df1['Exon&Intron'].str.len()

    #計算起始與終點位置方便標顏色
    #處理頭
    if df1.at[0,'temp'] != 'U' and df1.at[0,'temp'] !='L': #計算5'UTR
        df1.at[0,'Start'] = 1 #新增column
        df1.at[0,'End'] = df1.at[0,'Start'] + df1.at[0,'Length'] -1
        #計算Exon1(因為要含5'utr所以比較特別)
        df1.at[1,'Start'] = 1 
        df1.at[1,'End'] = df1.at[0,'Length'] + df1.at[1,'Length'] 
        df1.at[1,'Length'] = df1.at[0,'Length'] + df1.at[1,'Length']
    #處理沒有5'UTR的
    if df1.at[0,'temp'] == 'U' :
        df1.at[0,'Start'] = 1
        df1.at[0,'End'] = df1.at[0,'Start'] +df1.at[0,'Length'] -1
    #處理中間    
    for l in range(0,len(df1)): #計算Exon & Intron
        if df1.at[l,'Start'] !=1:
            df1.at[l,'Start'] = df1.at[l-1,'End'] + 1 #第2個的開始由第1個的結尾
            df1.at[l,'End'] = df1.at[l,'Start'] +df1.at[l,'Length'] -1
    #處理尾         
    if df1.at[len(df1)-1,'temp'] != 'U' and df1.at[len(df1)-1,'temp'] !='L': #計算3'UTR
        df1.at[len(df1)-1,'End'] = df1.at[len(df1)-2,'End'] + df1.at[len(df1)-1,'Length']
        df1.at[len(df1)-2,'End'] = df1.at[len(df1)-2,'End'] + df1.at[len(df1)-1,'Length'] #特別處理Exon因為要包含3'UTR
        df1.at[len(df1)-2,'Length'] = df1.at[len(df1)-2,'Length'] + df1.at[len(df1)-1,'Length']

    #for spliced
    transcription_spliced = '' #空的list
    upper_lower = ''  #空的list 用來記錄有幾次大小寫變化
    i = 0 

    while i <len(data2):
        #找Exon
        if data2[i].isupper(): #判斷大寫
            transcription_spliced  += data2[i] #把該個字元加進去
            if i == len(data2)-1: #用來停止while迴圈
                break
            elif data2[i+1].islower():
                transcription_spliced  += ','
                upper_lower += 'U,'
        #找Intron
        elif data2[i].islower(): #判斷小寫
            transcription_spliced  += data2[i]
            if i == len(data2)-1:
                break
            elif data2[i+1].isupper():
                transcription_spliced  += ','
                upper_lower += 'L,'
        i += 1 #逐個檢查
    #處理最後一個的大小寫
    if data2[len(data2)-2].isupper():
        upper_lower +='U'
    else:
        upper_lower +='L'


    #弄成DataFrame的形式
    df2 = pd.DataFrame(columns=['temp','Exon&Intron']) #建立兩個column的DataFrame
    df2['temp'] = upper_lower.split(',') #split 表示依照什麼東西來區別 因為是string
    df2['Exon&Intron'] = transcription_spliced.split(',')

    #處理5'UTR和3'UTR
    if df2.at[0,'temp'] == 'L': #開頭小寫表示有5'UTR
        df2.at[0,'temp'] = '5'
    
    if df2.at[len(df2)-1,'temp'] == 'L': #結尾小寫表示有3'UTR
        df2.at[len(df2)-1,'temp'] = '3'
    
    #刪掉Intron

    df3['Length'] = df3['Exon&Intron'].str.len()

    #計算起始與終點位置方便標顏色
    #處理頭
    if df3.at[0,'temp'] != 'U' and df3.at[0,'temp'] !='L': #計算5'UTR
        df3.at[0,'Start'] = 1 #新增column
        df3.at[0,'End'] = df3.at[0,'Start'] + df3.at[0,'Length'] -1
        #計算Exon1(因為要含5'utr所以比較特別)
        df3.at[1,'Start'] = 1 
        df3.at[1,'End'] = df3.at[0,'Length'] + df3.at[1,'Length'] 
        df3.at[1,'Length'] = df3.at[0,'Length'] + df3.at[1,'Length']
    #處理沒有5'UTR的
    if df3.at[0,'temp'] == 'U' :
        df3.at[0,'Start'] = 1
        df3.at[0,'End'] = df3.at[0,'Start'] +df3.at[0,'Length'] -1
    #處理中間    
    for l in range(0,len(df3)): #計算Exon & Intron
        if df3.at[l,'Start'] !=1:
            df3.at[l,'Start'] = df3.at[l-1,'End'] + 1 #第2個的開始由第1個的結尾
            df3.at[l,'End'] = df3.at[l,'Start'] +df3.at[l,'Length'] -1
    #處理尾         
    if df3.at[len(df3)-1,'temp'] != 'U' and df3.at[len(df3)-1,'temp'] !='L': #計算3'UTR
        df3.at[len(df3)-1,'End'] = df3.at[len(df3)-2,'End'] + df3.at[len(df3)-1,'Length']
        df3.at[len(df3)-2,'End'] = df3.at[len(df3)-2,'End'] + df3.at[len(df3)-1,'Length'] #特別處理Exon因為要包含3'UTR
        df3.at[len(df3)-2,'Length'] = df3.at[len(df3)-2,'Length'] + df3.at[len(df3)-1,'Length']
    #CDS
    CDS = {
        'temp': 'CDS',
        'Exon&Intron': df2.at[1,'Exon&Intron'],
        'Name': 'CDS',
        'Length':len(df2.at[1,'Exon&Intron']),
        'Start':len(df1.at[0,'Exon&Intron'])+1,
        'End':len(df2.at[0,'Exon&Intron'])+len(df2.at[1,'Exon&Intron'])}
    
    df4 = pd.DataFrame()
    df4 = df3
    CDS = pd.DataFrame([CDS])
    df3 = pd.concat([df3, CDS], ignore_index=True)


    specific_row_index = 1  # Replace with the desired index
    # Insert CDS at the specified row position in df3
    df4 = pd.concat([df4.iloc[:specific_row_index], CDS, df4.iloc[specific_row_index:]], ignore_index=True)
    
    position_to_insert = 2
    # part1 = df4.iloc[:position_to_insert]
    # part2 = df4.iloc[position_to_insert+1:]
    for i in range(0,len(df4)):
        if (df4.at[i,'Name'] == '3\'UTR'):
            row_to_insert = df4.iloc[i]
            df4 = pd.concat([df4.iloc[:position_to_insert], row_to_insert.to_frame().T, df4.iloc[position_to_insert:]], ignore_index=True)
            df4 = df4.drop(i)
    spliced_sp = ['Name','Start','End','Length']       
    spliced_sp = df4[spliced_sp]
    
    #for answer
    #unspliced
    unspliced =['Name','Start','End','Length']
    unspliced = df1[unspliced]
    print('unspliced:\n',unspliced)
    #spliced
    spliced = ['Name','Start','End','Length']
    spliced = df3[spliced]
    print('spliced:\n',spliced)
    
    #處理Codon
    a=df2.at[len(df2)-2,'Exon&Intron']
    i=0
    codon=''
    for k in range(0,len(a)):
        if a[k].isupper():
            codon += a[k]
            i += 1
            if (i%3) == 0 :
                codon += ','
            else:
                continue    
    #string to list
    codon = codon.split(',')
    del codon[len(codon)-1]
     
    #the dict of codon unspliced
    dict = {'TTT':'F', 'TTC':'F','TTA':'L','TTG':'L','CTT':'L','CTC':'L','CTA':'L','CTG':'L','ATT':'I','ATC':'I','ATA':'I','ATG':'M','GTT':'V','GTC':'V','GTA':'V','GTG':'V',
    'TCT':'S','TCC':'S','TCA':'S','TCG':'S','CCT':'P','CCC':'P','CCA':'P','CCG':'P','ACT':'T','ACC':'T','ACA':'T','ACG':'T','GCT':'A','GCC':'A','GCA':'A','GCG':'A','TAT':'Y','TAC':'Y','CAT':'H','CAC':'H','CAA':'Q','CAG':'Q','AAT':'N','AAC':'N','AAA':'K','AAG':'K','GAT':'D',
    'GAC':'D','GAA':'E','GAG':'E','TGT':'C','TGC':'C','TGG':'W','CGT':'R','CGC':'R','CGA':'R','CGG':'R','AGT':'S','AGC':'S','AGA':'R','AGG':'R','GGT':'G','GGC':'G','GGA':'G','GGG':'G','TAA':'','TGA':'','TAG':''}

    #output
    Stop = 0
    translation= []
    for values in codon :
        gene = dict.get(values)

        if Stop == 0:
            translation += gene        
        if gene == '' :
            Stop = 1         
        elif Stop == 1:
            print('end')
            Stop = 0
            break
    translation = ''.join(translation)
    print('translation:',translation)
else:
    print('hard')
    '''
    #靜態爬
    response = requests.get(
        "https://wormbase.org/rest/widget/transcript/F54H12.1c.3/sequences",
        headers={
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        })

    soup = BeautifulSoup(response.text, "html.parser")
    data= json.loads(str(soup))
    #處理變成dict
    strand = data["fields"]['unspliced_sequence_context']['data']['strand']
    '''
    #check + -
    if strand == "+" :
        transcriptdata1 = data["fields"]['unspliced_sequence_context']['data']['positive_strand']['features']
        transcriptdata2 = data["fields"]['spliced_sequence_context']['data']['positive_strand']['features']
    else:
        transcriptdata1= data["fields"]['unspliced_sequence_context']['data']['negative_strand']['features']
        transcriptdata2= data["fields"]['spliced_sequence_context']['data']['negative_strand']['features']

    def count_len(start: int,end: int):
        return end-start +1

    unspliced = pd.DataFrame(columns = ['type','start','stop'])
    spliced = pd.DataFrame(columns = ['type','start','stop'])

    for i in range(len(transcriptdata1)):
        unspliced.loc[i] = transcriptdata1[i]
    
    for i in range(len(transcriptdata2)):
        spliced.loc[i] = transcriptdata2[i]


    unspliced['length'] = unspliced.apply(lambda x: count_len(x['start'],x['stop']),axis=1)
    spliced['length'] = spliced.apply(lambda x: count_len(x['start'],x['stop']),axis=1)
    
    #重新命名
    #unspliced 
    count_Intron_u= 1
    count_Exon_u = 1
    for k in range(0,len(unspliced)):
        if unspliced.at[k,'type'] == 'intron':
            unspliced.at[k,'type'] = 'Intron' + str(count_Intron_u)
            count_Intron_u+= 1
        elif unspliced.at[k,'type'] == 'exon':
            unspliced.at[k,'type'] = 'Exon' + str(count_Exon_u)
            count_Exon_u += 1
        elif unspliced.at[k,'type'] == 'five_prime_UTR':
            unspliced.at[k,'type'] = '5\'UTR'
        elif unspliced.at[k,'type'] == 'three_prime_UTR':
            unspliced.at[k,'type'] = '3\'UTR'
    #spliced
    count_Exon_s = 1
    for k in range(0,len(spliced)):
        if spliced.at[k,'type'] == 'exon':
            spliced.at[k,'type'] = 'Exon' + str(count_Exon_s)
            count_Exon_s += 1
        elif spliced.at[k,'type'] == 'five_prime_UTR':
            spliced.at[k,'type'] = '5\'UTR'
        elif spliced.at[k,'type'] == 'three_prime_UTR':
            spliced.at[k,'type'] = '3\'UTR'
  
    #for answer
    #unspliced       
    print('unspliced:\n',unspliced)

    #spliced
    print('spliced:\n',spliced)
    

    translation = data['fields']['protein_sequence']['data']['sequence']
    print('translation:',translation)
        

easy
unspliced:
       Name   Start     End  Length
0    5'UTR     1.0   131.0     131
1    Exon1     1.0   191.0     191
2  Intron1   192.0  2028.0    1837
3    Exon2  2029.0  2359.0     331
4  Intron2  2360.0  3192.0     833
5    Exon3  3193.0  3326.0     134
6  Intron3  3327.0  3388.0      62
7    Exon4  3389.0  3600.0     212
8    3'UTR  3545.0  3600.0      56
spliced:
     Name  Start    End  Length
0  5'UTR    1.0  131.0     131
1  Exon1    1.0  191.0     191
2  Exon2  192.0  522.0     331
3  Exon3  523.0  656.0     134
4  Exon4  657.0  868.0     212
5  3'UTR  813.0  868.0      56
6    CDS  132.0  812.0     681
translation: MSNVAKSYKSSDPVIAYCSEHTTIQSPLQAELLKETLDNAPMSMMLGAPEVLTFGQNFIRSFGGRRVIDVGTYTGASALAWALAVPDDGEVFTFDIDHTNYKKFGVPIISKCEKTFKKIRAIEGSALDGLDKLIADGQSGTFDFAFIDADKVNYSNYYDRAVTLLRKGGVIFVDNSLWNGSVCDPARRTEPSTVAIHQANDKIFKDDRTYSALINLGDGTHIAFKK


In [291]:
print(spliced_sp)

    Name  Start    End Length
0  5'UTR    1.0  131.0    131
1    CDS  132.0  812.0    681
2  3'UTR  813.0  868.0     56
3  Exon1    1.0  191.0    191
4  Exon2  192.0  522.0    331
5  Exon3  523.0  656.0    134
6  Exon4  657.0  868.0    212
7  3'UTR  813.0  868.0     56


In [292]:
unspliced = unspliced.values.tolist()
spliced = spliced.values.tolist()

unspliced_list = []
spliced_list = []
translation_list = []



for i in range(len(unspliced)):
        unspliced_list.append({"Name":unspliced[i][0]})
        unspliced_list[i]["Start"] = unspliced[i][1]
        unspliced_list[i]["End"] = unspliced[i][2]
        unspliced_list[i]["Length"] = unspliced[i][3]
        

for i in range(len(spliced)):
        spliced_list.append({"Name":spliced[i][0]})
        spliced_list[i]["Start"] = spliced[i][1]
        spliced_list[i]["End"] = spliced[i][2]
        spliced_list[i]["Length"] = spliced[i][3]

translation_list.append({"translation":translation}) 

In [293]:
unspliced_index = []
exon_type_u = 0
if ('3\'UTR' in unspliced):
    if ('5\'UTR' in unspliced):
        print('1')
        for i in range(0,len(unspliced)):
            if (unspliced[i][0] == '5\'UTR'):
                for j in range(int(unspliced[i][1]-1),int(unspliced[i][2])):
                    unspliced_index.append(1)
            elif('Exon' in unspliced[i][0]):
                    if(exon_type_u == 0 ) :
                        if(unspliced[i+1][0] == '3\'UTR'):
                            for j in range(int(unspliced[i-1][2]),int(unspliced[i+1][1]-1)):
                                unspliced_index.append(2)
                                exon_type_u = 1 
                        else:
                            for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                                unspliced_index.append(2)
                                exon_type_u = 1
                    elif (exon_type_u == 1) :
                        if(unspliced[i+1][0] == '3\'UTR'):
                            for j in range(int(unspliced[i-1][2]),int(unspliced[i+1][1]-1)):
                                unspliced_index.append(3) 
                                exon_type_u =0
                        else:
                            for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                                unspliced_index.append(3)
                                exon_type_u =0                
            elif('Intron' in unspliced[i][0]):
                    if(unspliced[i+1][0] == '3\'UTR'):
                        for j in range(int(unspliced[i-1][2]),int(unspliced[i+1][1])):
                            unspliced_index.append(4) 
                    else:
                        for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                            unspliced_index.append(4)
            elif(unspliced[i][0] == '3\'UTR'):
                for j in range(int(unspliced[i][1]-1),int(unspliced[i][2])):
                    unspliced_index.append(5)
    else:
        print('2')
        for i in range(0,len(unspliced)):
            if('Exon' in unspliced[i][0]):
                    if(exon_type_u == 0 ) :
                        if(unspliced[i+1][0] == '3\'UTR'):
                            for j in range(int(unspliced[i-1][2]),int(unspliced[i+1][1]-1)):
                                unspliced_index.append(2)
                                exon_type_u = 1 
                        else:
                            for j in range(int(unspliced[i][1]-1),int(unspliced[i][2])):
                                unspliced_index.append(2)
                                exon_type_u = 1
                    elif (exon_type_u == 1) :
                        if(unspliced[i+1][0] == '3\'UTR'):
                            for j in range(int(unspliced[i-1][2]),int(unspliced[i+1][1]-1)):
                                unspliced_index.append(3) 
                                exon_type_u =0
                        else:
                            for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                                unspliced_index.append(3)
                                exon_type_u =0                
            elif('Intron' in unspliced[i][0]):
                    if(unspliced[i+1][0] == '3\'UTR'):
                        for j in range(int(unspliced[i-1][2]),int(unspliced[i+1][1])):
                            unspliced_index.append(4) 
                    else:
                        for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                            unspliced_index.append(4)
            elif(unspliced[i][0] == '3\'UTR'):
                for j in range(int(unspliced[i][1]-1),int(unspliced[i][2])):
                    unspliced_index.append(5)
else:
    
    if ('5\'UTR' in unspliced):
        print('3')
        for i in range(0,len(unspliced)):
            if (unspliced[i][0] == '5\'UTR'):
                for j in range(int(unspliced[i][1]-1),int(unspliced[i][2])):
                    unspliced_index.append(1)
            elif('Exon' in unspliced[i][0]):
                    if(exon_type_u == 0 ) :
                        for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                            unspliced_index.append(2)
                            exon_type_u = 1
                    elif (exon_type_u == 1) :
                        for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                            unspliced_index.append(3)
                            exon_type_u =0                
            elif('Intron' in unspliced[i][0]):
                    # if(unspliced[i+1][0] == '3\'UTR'):
                    #     for j in range(int(unspliced[i-1][2]),int(unspliced[i+1][1])):
                    #         unspliced_index.append(4) 
                    # else:
                    for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                        unspliced_index.append(4)
            # elif(unspliced[i][0] == '3\'UTR'):
            #     for j in range(int(unspliced[i][1]-1),int(unspliced[i][2])):
            #         unspliced_index.append(5)
    else:
        print('4')
        for i in range(0,len(unspliced)):
            if('Exon' in unspliced[i][0]):
                if(exon_type_u == 0 ) :
                    for j in range(int(unspliced[i][1]-1),int(unspliced[i][2])):
                        unspliced_index.append(2)
                        exon_type_u = 1
                elif (exon_type_u == 1) :
                    for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                        unspliced_index.append(3)
                        exon_type_u =0                
            elif('Intron' in unspliced[i][0]):
                for j in range(int(unspliced[i-1][2]),int(unspliced[i][2])):
                    unspliced_index.append(4)

print(unspliced_index)
print(len(unspliced_index))
print(unspliced_index.count(3))

4
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4

In [294]:
spliced_index = []
exon_type_s = 0
if ('3\'UTR' in unspliced):
    if ('5\'UTR' in unspliced):
        for i in range(0,len(spliced)):
            if (spliced[i][0] == '5\'UTR'):
                for j in range(int(spliced[i][1]-1),int(spliced[i][2])):
                    spliced_index.append(1)
            elif('Exon' in spliced[i][0]):
                    if(exon_type_s == 0 ) :
                        if(spliced[i+1][0] == '3\'UTR'):
                            for j in range(int(spliced[i-1][2]),int(spliced[i+1][1]-1)):
                                spliced_index.append(2)
                                exon_type_s = 1 
                        else:
                            for j in range(int(spliced[i-1][2]),int(spliced[i][2])):
                                spliced_index.append(2)
                                exon_type_s = 1
                    elif (exon_type_s == 1) :
                        if(spliced[i+1][0] == '3\'UTR'):
                            for j in range(int(spliced[i-1][2]),int(spliced[i+1][1]-1)):
                                spliced_index.append(3) 
                                exon_type_s =0
                        else:
                            for j in range(int(spliced[i-1][2]),int(spliced[i][2])):
                                spliced_index.append(3)
                                exon_type_s =0                
            elif(spliced[i][0] == '3\'UTR'):
                for j in range(int(spliced[i][1]-1),int(spliced[i][2])):
                    spliced_index.append(5)
    else:
        for i in range(0,len(spliced)):
            if('Exon' in spliced[i][0]):
                if(exon_type_s == 0 ) :
                    if(spliced[i+1][0] == '3\'UTR'):
                        for j in range(int(spliced[i-1][2]),int(spliced[i+1][1]-1)):
                            spliced_index.append(2)
                            exon_type_s = 1 
                    else:
                        for j in range(int(spliced[i][1]-1),int(spliced[i][2])):
                            spliced_index.append(2)
                            exon_type_s = 1
                elif (exon_type_s == 1) :
                    if(spliced[i+1][0] == '3\'UTR'):
                        for j in range(int(spliced[i-1][2]),int(spliced[i+1][1]-1)):
                            spliced_index.append(3) 
                            exon_type_s =0
                    else:
                        for j in range(int(spliced[i-1][2]),int(spliced[i][2])):
                            spliced_index.append(3)
                            exon_type_s =0                
            elif(spliced[i][0] == '3\'UTR'):
                for j in range(int(spliced[i][1]-1),int(spliced[i][2])):
                    spliced_index.append(5)
else:
    if ('5\'UTR' in unspliced):
        for i in range(0,len(spliced)):
            if (spliced[i][0] == '5\'UTR'):
                for j in range(int(spliced[i][1]-1),int(spliced[i][2])):
                    spliced_index.append(1)
            elif('Exon' in spliced[i][0]):
                    if(exon_type_s == 0 ) :
                        for j in range(int(spliced[i-1][2]),int(spliced[i][2])):
                            spliced_index.append(2)
                            exon_type_s = 1
                    elif (exon_type_s == 1) :
                        for j in range(int(spliced[i-1][2]),int(spliced[i][2])):
                            spliced_index.append(3)
                            exon_type_s =0                
    else:
        for i in range(0,len(spliced)):
            if('Exon' in spliced[i][0]):
                if(exon_type_s == 0 ) :
                    for j in range(int(spliced[i][1]-1),int(spliced[i][2])):
                        spliced_index.append(2)
                        exon_type_s = 1
                elif (exon_type_s == 1) :
                    for j in range(int(spliced[i-1][2]),int(spliced[i][2])):
                        spliced_index.append(3)
                        exon_type_s =0     
print(spliced_index)
print(len(spliced_index))
print(spliced_index.count(3))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [295]:
print(unspliced)
print(spliced)
if ('3\'UTR' in unspliced):
    print('yes')
else:
    print('no')

[["5'UTR", 1.0, 131.0, 131], ['Exon1', 1.0, 191.0, 191], ['Intron1', 192.0, 2028.0, 1837], ['Exon2', 2029.0, 2359.0, 331], ['Intron2', 2360.0, 3192.0, 833], ['Exon3', 3193.0, 3326.0, 134], ['Intron3', 3327.0, 3388.0, 62], ['Exon4', 3389.0, 3600.0, 212], ["3'UTR", 3545.0, 3600.0, 56]]
[["5'UTR", 1.0, 131.0, 131], ['Exon1', 1.0, 191.0, 191], ['Exon2', 192.0, 522.0, 331], ['Exon3', 523.0, 656.0, 134], ['Exon4', 657.0, 868.0, 212], ["3'UTR", 813.0, 868.0, 56], ['CDS', 132.0, 812.0, 681]]
no


In [296]:
unspliced_d3 = []
exon_type_d3_u = 0
for i in range(0,len(unspliced)):
    if (unspliced[i][0] == '5\'UTR'):
        unspliced_d3.append({"Type":'5\'UTR'})
        unspliced_d3[i]["start"] = unspliced[i][1]
        unspliced_d3[i]["end"] = unspliced[i][2]
        unspliced_d3[i]["x1"] = unspliced[i][1]
        unspliced_d3[i]["y1"] = 50
        unspliced_d3[i]["x2"] = unspliced[i][2]
        unspliced_d3[i]["y2"] = 50
        unspliced_d3[i]["color"] = "gray"
    elif('Exon' in unspliced[i][0]):
            if(exon_type_d3_u == 0 ) :
                unspliced_d3.append({"Type":'Exon'})
                unspliced_d3[i]["start"] = unspliced[i][1]
                unspliced_d3[i]["end"] = unspliced[i][2]
                unspliced_d3[i]["x1"] = unspliced[i][1]
                unspliced_d3[i]["y1"] = 50
                unspliced_d3[i]["x2"] = unspliced[i][2]
                unspliced_d3[i]["y2"] = 50
                unspliced_d3[i]["color"] = "yellow"
                exon_type_d3_u = 1
            elif (exon_type_d3_u == 1) :
                unspliced_d3.append({"Type":'Exon'})
                unspliced_d3[i]["start"] = unspliced[i][1]
                unspliced_d3[i]["end"] = unspliced[i][2]
                unspliced_d3[i]["x1"] = unspliced[i][1]
                unspliced_d3[i]["y1"] = 50
                unspliced_d3[i]["x2"] = unspliced[i][2]
                unspliced_d3[i]["y2"] = 50
                unspliced_d3[i]["color"] = "orange" 
                exon_type_d3_u = 0             
    elif('Intron' in unspliced[i][0]):
        unspliced_d3.append({"Type":'Intron'})
        unspliced_d3[i]["start"] = unspliced[i][1]
        unspliced_d3[i]["end"] = unspliced[i][2]
        unspliced_d3[i]["x1"] = unspliced[i][1]
        unspliced_d3[i]["y1"] = 50
        unspliced_d3[i]["x2"] = unspliced[i][2]
        unspliced_d3[i]["y2"] = 50
        unspliced_d3[i]["color"] = "GhostWhite"          
    elif(unspliced[i][0] == '3\'UTR'):
        unspliced_d3.append({"Type":'3\'UTR'})
        unspliced_d3[i]["start"] = unspliced[i][1]
        unspliced_d3[i]["end"] = unspliced[i][2]
        unspliced_d3[i]["x1"] = unspliced[i][1]
        unspliced_d3[i]["y1"] = 50
        unspliced_d3[i]["x2"] = unspliced[i][2]
        unspliced_d3[i]["y2"] = 50
        unspliced_d3[i]["color"] = "gray"          

In [297]:
print(unspliced_d3)

[{'Type': "5'UTR", 'start': 1.0, 'end': 131.0, 'x1': 1.0, 'y1': 50, 'x2': 131.0, 'y2': 50, 'color': 'gray'}, {'Type': 'Exon', 'start': 1.0, 'end': 191.0, 'x1': 1.0, 'y1': 50, 'x2': 191.0, 'y2': 50, 'color': 'yellow'}, {'Type': 'Intron', 'start': 192.0, 'end': 2028.0, 'x1': 192.0, 'y1': 50, 'x2': 2028.0, 'y2': 50, 'color': 'GhostWhite'}, {'Type': 'Exon', 'start': 2029.0, 'end': 2359.0, 'x1': 2029.0, 'y1': 50, 'x2': 2359.0, 'y2': 50, 'color': 'orange'}, {'Type': 'Intron', 'start': 2360.0, 'end': 3192.0, 'x1': 2360.0, 'y1': 50, 'x2': 3192.0, 'y2': 50, 'color': 'GhostWhite'}, {'Type': 'Exon', 'start': 3193.0, 'end': 3326.0, 'x1': 3193.0, 'y1': 50, 'x2': 3326.0, 'y2': 50, 'color': 'yellow'}, {'Type': 'Intron', 'start': 3327.0, 'end': 3388.0, 'x1': 3327.0, 'y1': 50, 'x2': 3388.0, 'y2': 50, 'color': 'GhostWhite'}, {'Type': 'Exon', 'start': 3389.0, 'end': 3600.0, 'x1': 3389.0, 'y1': 50, 'x2': 3600.0, 'y2': 50, 'color': 'orange'}, {'Type': "3'UTR", 'start': 3545.0, 'end': 3600.0, 'x1': 3545.0, 

In [298]:
spliced_d3 = []
exon_type_d3_s = 0
for i in range(0,len(spliced)):
    if (spliced[i][0] == '5\'UTR'):
        spliced_d3.append({"x1":(spliced[i][1]/10)+360})
        spliced_d3[i]["y1"] = 50
        spliced_d3[i]["x2"] = (spliced[i][2]/10)+360
        spliced_d3[i]["y2"] = 50
        spliced_d3[i]["color"] = "gray"
    elif('Exon' in spliced[i][0]):
            if(exon_type_d3_s == 0 ) :
                spliced_d3.append({"x1":(spliced[i][1]/10)+360})
                spliced_d3[i]["y1"] = 50
                spliced_d3[i]["x2"] = (spliced[i][2]/10)+360
                spliced_d3[i]["y2"] = 50
                spliced_d3[i]["color"] = "yellow"
                exon_type_d3_s = 1
            elif (exon_type_d3_s == 1) :
                spliced_d3.append({"x1":(spliced[i][1]/10)+360})
                spliced_d3[i]["y1"] = 50
                spliced_d3[i]["x2"] = (spliced[i][2]/10)+360
                spliced_d3[i]["y2"] = 50
                spliced_d3[i]["color"] = "orange" 
                exon_type_d3_s = 0               
    elif(spliced[i][0] == '3\'UTR'):
        spliced_d3.append({"x1":(spliced[i][1]/10)+360})
        spliced_d3[i]["y1"] = 50
        spliced_d3[i]["x2"] = (spliced[i][2]/10)+360
        spliced_d3[i]["y2"] = 50
        spliced_d3[i]["color"] = "gray"  

In [299]:
print(spliced_d3)

[{'x1': 360.1, 'y1': 50, 'x2': 373.1, 'y2': 50, 'color': 'gray'}, {'x1': 360.1, 'y1': 50, 'x2': 379.1, 'y2': 50, 'color': 'yellow'}, {'x1': 379.2, 'y1': 50, 'x2': 412.2, 'y2': 50, 'color': 'orange'}, {'x1': 412.3, 'y1': 50, 'x2': 425.6, 'y2': 50, 'color': 'yellow'}, {'x1': 425.7, 'y1': 50, 'x2': 446.8, 'y2': 50, 'color': 'orange'}, {'x1': 441.3, 'y1': 50, 'x2': 446.8, 'y2': 50, 'color': 'gray'}]
